In [ ]:
import os
import cv2
import threading
import ipywidgets as widgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

In [ ]:
#left_camera
camera0 = CSICamera(capture_device=0, width=224, height=224)
#right_camera
camera1 = CSICamera(capture_device=1, width=224, height=224)

camera0.running = True
camera1.running = True
print("camera created")

In [2]:

#图像保存目录
left_images_path = 'left_images'
right_images_path = 'right_images'

#图像编号
image_counter = 1

def save_image(camera, path, prefix):
    global image_counter
    image_name = f"{image_counter:03d}_{prefix}.jpg"
    image_path = os.path.join(path, image_name)
    if not os.path.exists(path):
        os.makedirs(path)
    cv2.imwrite(image_path, camera.value)
    print(f"{image_name} saved to {path}")

In [ ]:
#图像预览
image_widget0 = widgets.Image(format='jpeg', width=224, height=224)
image_widget1 = widgets.Image(format='jpeg', width=224, height=224)

# 将图像数据与窗口关联
camera_link0 = traitlets.dlink((camera0, 'value'), (image_widget0, 'value'), transform=bgr8_to_jpeg)
camera_link1 = traitlets.dlink((camera1, 'value'), (image_widget1, 'value'), transform=bgr8_to_jpeg)

# 创建一个水平布局，用于显示左右相机图像
hbox = widgets.HBox([image_widget0, image_widget1])
hbox = widgets.HBox([image_widget0])

# 显示布局
display(hbox)

In [ ]:
def capture():
    global image_counter
    threading.Thread(target=save_image, args=(camera0, left_images_path, "left")).start()
    threading.Thread(target=save_image, args=(camera1, right_images_path, "right")).start()
    image_counter += 1

def capture_on_click(b):
    capture()
    
# 在 Jupyter Lab 中创建一个按钮
capture_button = widgets.Button(description="Capture")
capture_button.on_click(capture_on_click)
display(capture_button)

In [ ]:
data_collection_widget = widgets.VBox([
    widgets.HBox([image_widget0]),
    capture_button
])

display(data_collection_widget)

In [6]:
!zip -r -q left_images.zip left_images

In [7]:
!zip -r -q right_images.zip right_images